In [1]:
import os
import json
import gridSearch as g
import math
import batching as b
import modularized as m
import convL
import convo as c
import numpy as np
from tensorflow import keras
import tensorflow as tf
import gridS
import funcy as fy
import gin
import poolingL
from ogb.nodeproppred import NodePropPredDataset
from ogb.graphproppred import GraphPropPredDataset
from sklearn.model_selection import ParameterGrid
import experiments as e
%reload_ext autoreload
%autoreload 2


2023-05-12 11:27:26.322108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 11:27:26.952284: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-12 11:27:26.952313: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-12 11:27:28.911511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
dataset1 = GraphPropPredDataset(name = 'ogbg-molhiv')

In [3]:
dataset_arxiv = NodePropPredDataset(name='ogbn-arxiv')


In [4]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}

override_conv_type = 'gin'  # None

if override_conv_type is not None:
    param_grid['convo_type'] = [override_conv_type]

small_sample = list(ParameterGrid(param_grid))[0:1]
sample = list(ParameterGrid(param_grid))[0:5]
big_sample = list(ParameterGrid(param_grid))[0:20]
small_sample


[{'activation': 'relu',
  'convo_type': 'gin',
  'learning_rate': 0.0001,
  'num_layers': 1,
  'optimizer': 'Adam',
  'probability': 0.3,
  'regularization': 'DropOut',
  'units': 32}]

In [5]:
ds = GraphPropPredDataset('ogbg-molhiv')

In [8]:
g.create_grid_models(big_sample,'ogbg-molhiv')

Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropOut units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropOut units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:NodeSampling units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:NodeSampling units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropEdge units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropEdge units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer

1

In [28]:
gridS.train_and_evaluate(small_sample, 'ogbg-molhiv')
for dataset in datasets: 
    gridS.train_and_evaluate(sample, 'ogbg-molpcba')

Evaluating hyperparameter configuration 0 for dataset ogbg-molhiv.
138/138 [==============================] - 1s 3ms/step


In [12]:
import os
import json

# Define the base directory where the experiment results will be saved
base_dir = "experiment_results"

# Define a list of dataset names
datasets = ["dataset_A", "dataset_B", "dataset_C"]

# Define a dictionary of hyperparameters for each dataset
hyperparams_dict = {
    "dataset_A": {
        "param1": 0.5,
        "param2": 10,
        "param3": "string"
    },
    "dataset_B": {
        "param1": 0.3,
        "param2": 5,
        "param3": "another string"
    },
    "dataset_C": {
        "param1": 0.8,
        "param2": 20,
        "param3": "third string"
    }
}

# Define a list of hyperparameter configurations to test for each dataset
hp_configs = {
    "dataset_A": ["hpconfig_0", "hpconfig_1", "hpconfig_2"],
    "dataset_B": ["hpconfig_0", "hpconfig_1"],
    "dataset_C": ["hpconfig_0"]
}

# Define the number of times to repeat each hyperparameter configuration
num_repeats = 3

# Iterate over each dataset
for dataset in datasets:
    # Define the directory path for the current dataset
    dataset_dir = os.path.join(base_dir, dataset)
    
    # Create the directory for the current dataset
    os.makedirs(dataset_dir, exist_ok=True)
    
    # Define the path for the hyperparameters JSON file
    hyperparams_path = os.path.join(dataset_dir, "hyperparams.json")
    
    # Write the hyperparameters dictionary to the JSON file
    with open(hyperparams_path, "w") as f:
        json.dump(hyperparams_dict[dataset], f)
    
    # Iterate over each hyperparameter configuration for the current dataset
    for hp_config in hp_configs[dataset]:
        # Define the directory path for the current hyperparameter configuration
        hp_config_dir = os.path.join(dataset_dir, hp_config)
        
        # Create the directory for the current hyperparameter configuration
        os.makedirs(hp_config_dir, exist_ok=True)
        
        # Iterate over the number of repeats for the current hyperparameter configuration
        for repeat in range(num_repeats):
            # Define the filename for the current repeat
            filename = f"repeat_{repeat}.json"
            
            # Define the path for the current JSON file
            json_path = os.path.join(hp_config_dir, filename)
            
            # Define the dictionary to write to the current JSON file
            data = {
                "dataset": dataset,
                "hyperparams": hyperparams_dict[dataset],
                "hp_config": hp_config,
                "repeat": repeat
            }
            
            # Write the dictionary to the current JSON file
            with open(json_path, "w") as f:
                json.dump(data, f)
